In [1]:
# Pandas for managing datasets
import pandas as pd

In [2]:
# Display floats with 2 decimal places
pd.options.display.float_format = '{:,.2f}'.format
 
# Expand display limits
pd.options.display.max_rows = 200
pd.options.display.max_columns = 100

In [3]:
# Read BNC2 sample dataset
df = pd.read_csv('BNC2_sample.csv',
                 names=['Code', 'Date', 'Open', 'High', 'Low', 
                        'Close', 'Volume', 'VWAP', 'TWAP'])
 
# Display first 5 observations
df.head()

,Code,Date,Open,High,Low,Close,Volume,VWAP,TWAP
0,GWA_BTC,2014-04-01,467.28,488.62,467.28,479.56,"74,776.48",482.76,482.82
1,GWA_BTC,2014-04-02,479.20,494.30,431.32,437.08,"114,052.96",460.19,465.93
2,GWA_BTC,2014-04-03,437.33,449.74,414.41,445.60,"91,415.08",432.29,433.28
3,GWA_BTC,2014-04-04,445.18,456.10,429.16,449.81,"51,147.27",443.46,443.93
4,GWA_BTC,2014-04-05,450.08,464.09,445.16,461.70,"28,449.19",452.53,452.95


In [4]:
print( df.Code.unique() )

['GWA_BTC' 'GWA_ETH' 'GWA_LTC' 'GWA_XLM' 'GWA_XRP' 'MWA_BTC_CNY'
 'MWA_BTC_EUR' 'MWA_BTC_GBP' 'MWA_BTC_JPY' 'MWA_BTC_USD' 'MWA_ETH_CNY'
 'MWA_ETH_EUR' 'MWA_ETH_GBP' 'MWA_ETH_JPY' 'MWA_ETH_USD' 'MWA_LTC_CNY'
 'MWA_LTC_EUR' 'MWA_LTC_GBP' 'MWA_LTC_JPY' 'MWA_LTC_USD' 'MWA_XLM_CNY'
 'MWA_XLM_EUR' 'MWA_XLM_USD' 'MWA_XRP_CNY' 'MWA_XRP_EUR' 'MWA_XRP_GBP'
 'MWA_XRP_JPY' 'MWA_XRP_USD']


In [5]:
gwa_codes = [code for code in df.Code.unique() if 'GWA_' in code]

df = df[df.Code.isin(gwa_codes)]

print(len(df))

6309


In [6]:
df.Code.unique()

array(['GWA_BTC', 'GWA_ETH', 'GWA_LTC', 'GWA_XLM', 'GWA_XRP'],
      dtype=object)

In [34]:
df.head()

,Code,Date,Open,High,Low,Close,Volume,VWAP,TWAP
0,GWA_BTC,2014-04-01,467.28,488.62,467.28,479.56,"74,776.48",482.76,482.82
1,GWA_BTC,2014-04-02,479.20,494.30,431.32,437.08,"114,052.96",460.19,465.93
2,GWA_BTC,2014-04-03,437.33,449.74,414.41,445.60,"91,415.08",432.29,433.28
3,GWA_BTC,2014-04-04,445.18,456.10,429.16,449.81,"51,147.27",443.46,443.93
4,GWA_BTC,2014-04-05,450.08,464.09,445.16,461.70,"28,449.19",452.53,452.95


In [7]:
pivoted_df = df.pivot(index='Date',columns = 'Code',values = 'VWAP')

pivoted_df.tail()

Code,GWA_BTC,GWA_ETH,GWA_LTC,GWA_XLM,GWA_XRP
Date,,,,,
2018-01-19,"11,826.36","1,068.45",195.00,0.51,1.82
2018-01-20,"13,062.68","1,158.71",207.58,0.52,1.75
2018-01-21,"12,326.23","1,108.90",197.36,0.48,1.55
2018-01-22,"11,397.52","1,038.21",184.92,0.47,1.43
2018-01-23,"10,921.00",992.05,176.95,0.47,1.42


In [15]:
# Calculate returns over 7 days prior
delta_7 = pivoted_df / pivoted_df.shift(7) - 1.0
 
# Display examples
delta_7.tail()

Code,GWA_BTC,GWA_ETH,GWA_LTC,GWA_XLM,GWA_XRP
Date,,,,,
2018-01-19,-0.18,-0.17,-0.18,-0.21,-0.22
2018-01-20,-0.13,-0.19,-0.18,-0.23,-0.29
2018-01-21,-0.15,-0.20,-0.22,-0.22,-0.30
2018-01-22,-0.21,-0.24,-0.24,-0.25,-0.32
2018-01-23,-0.11,-0.12,-0.13,-0.02,-0.04


In [11]:
delta_dict = {}
for offset in [7,14,21,28]:
    delta_dict['delta_{}'.format(offset)] = pivoted_df / pivoted_df.shift(offset)

In [13]:
delta_dict.keys()

dict_keys(['delta_7', 'delta_14', 'delta_21', 'delta_28'])

In [16]:
# Melt delta_7 returns
melted_7 = delta_7.reset_index().melt(id_vars=['Date'], value_name='delta_7')
 
# Melted dataframe examples
melted_7.tail()

,Date,Code,delta_7
6965,2018-01-19,GWA_XRP,-0.22
6966,2018-01-20,GWA_XRP,-0.29
6967,2018-01-21,GWA_XRP,-0.30
6968,2018-01-22,GWA_XRP,-0.32
6969,2018-01-23,GWA_XRP,-0.04


In [17]:
pd.unique(melted_7.Code)

array(['GWA_BTC', 'GWA_ETH', 'GWA_LTC', 'GWA_XLM', 'GWA_XRP'],
      dtype=object)

In [18]:
# Melt all the delta dataframes and store in list
melted_dfs = []
for key, delta_df in delta_dict.items():
    melted_dfs.append( delta_df.reset_index().melt(id_vars=['Date'], value_name=key) )

In [19]:
# Calculate 7-day returns after the date
return_df = pivoted_df.shift(-7) / pivoted_df - 1.0
 
# Melt the return dataset and append to list
melted_dfs.append( return_df.reset_index().melt(id_vars=['Date'], value_name='return_7') )

In [21]:
from functools import reduce

In [22]:
# Grab features from original dataset
base_df = df[['Date', 'Code', 'Volume', 'VWAP']]
 
# Create a list with all the feature dataframes
feature_dfs = [base_df] + melted_dfs

In [33]:
# Reduce-merge features into analytical base table
abt = reduce(lambda left,right: pd.merge(left,right,on=['Date', 'Code']), feature_dfs)
 
# Display examples from the ABT
abt.head(30)

,Date,Code,Volume,VWAP,delta_7,delta_14,delta_21,delta_28,return_7
0,2014-04-01,GWA_BTC,"74,776.48",482.76,nan,nan,nan,nan,-0.06
1,2014-04-02,GWA_BTC,"114,052.96",460.19,nan,nan,nan,nan,-0.02
2,2014-04-03,GWA_BTC,"91,415.08",432.29,nan,nan,nan,nan,-0.07
3,2014-04-04,GWA_BTC,"51,147.27",443.46,nan,nan,nan,nan,-0.09
4,2014-04-05,GWA_BTC,"28,449.19",452.53,nan,nan,nan,nan,-0.05
5,2014-04-06,GWA_BTC,"21,720.64",460.92,nan,nan,nan,nan,-0.10
6,2014-04-07,GWA_BTC,"34,949.20",455.32,nan,nan,nan,nan,-0.01
7,2014-04-08,GWA_BTC,"24,290.14",453.96,0.94,nan,nan,nan,0.07
8,2014-04-09,GWA_BTC,"30,135.25",450.43,0.98,nan,nan,nan,0.15
9,2014-04-10,GWA_BTC,"149,892.25",401.84,0.93,nan,nan,nan,0.26


In [35]:
# Create 'month' feature
abt['month'] = abt.Date.apply(lambda x: x[:7])

In [38]:
gb_df = abt.groupby(['Code', 'month']).first()
gb_df.reset_index().head()

Date     Volume   VWAP  delta_7  delta_14  delta_21  \
Code    month                                                                
GWA_BTC 2014-04  2014-04-01  74,776.48 482.76     0.94      1.01      1.03   
        2014-05  2014-05-01  29,669.84 455.13     0.93      0.90      1.13   
        2014-06  2014-06-01 102,628.09 646.77     1.14      1.44      1.46   
        2014-07  2014-07-01 150,917.11 650.25     1.09      1.08      1.00   
        2014-08  2014-08-01 196,450.76 591.68     0.98      0.95      0.95   

                 delta_28  return_7  
Code    month                        
GWA_BTC 2014-04      0.92     -0.06  
        2014-05      1.05     -0.03  
        2014-06      1.49      0.02  
        2014-07      0.98     -0.05  
        2014-08      0.93      0.01

In [39]:
gb_df.reset_index().head().tail()

,Code,month,Date,Volume,VWAP,delta_7,delta_14,delta_21,delta_28,return_7
0,GWA_BTC,2014-04,2014-04-01,"74,776.48",482.76,0.94,1.01,1.03,0.92,-0.06
1,GWA_BTC,2014-05,2014-05-01,"29,669.84",455.13,0.93,0.90,1.13,1.05,-0.03
2,GWA_BTC,2014-06,2014-06-01,"102,628.09",646.77,1.14,1.44,1.46,1.49,0.02
3,GWA_BTC,2014-07,2014-07-01,"150,917.11",650.25,1.09,1.08,1.00,0.98,-0.05
4,GWA_BTC,2014-08,2014-08-01,"196,450.76",591.68,0.98,0.95,0.95,0.93,0.01
